This Spark notebook will answer the questions of the Amadeus Challenge (bookings + searches) using Spark instead of python pandas.

# RDD's

In [1]:
import org.apache.spark.storage.StorageLevel._

# 1) First exercise: count the number of lines in Python for each file

Create manually the SparkContext (this is not needed, by default sc is loaded). Indeed if I try this I have problems...

## Read files
Define paths and load RDDs on searches and bookings, save first line as header then remove first lines from RDD.

In [2]:
val root_path: String = "/media/sf_VirtualFolder/amadeus/";//where to find files
val searches_path: String = root_path + "searches.csv.bz2";
val bookings_path: String = root_path + "bookings.csv.bz2";

root_path = /media/sf_VirtualFolder/amadeus/
searches_path = /media/sf_VirtualFolder/amadeus/searches.csv.bz2
bookings_path = /media/sf_VirtualFolder/amadeus/bookings.csv.bz2


/media/sf_VirtualFolder/amadeus/bookings.csv.bz2

In [3]:
val searches = sc.textFile(searches_path)//Load RDD 1
val bookings = sc.textFile(bookings_path)//Load RDD 2

val headerColumns_searches = searches.first().split("\\^").to[List]
val head_searches = sc.textFile(searches_path).first()

val headerColumns_bookings = bookings.first().split("\\^").to[List]
val head_bookings = sc.textFile(bookings_path).first()

searches = /media/sf_VirtualFolder/amadeus/searches.csv.bz2 MapPartitionsRDD[1] at textFile at <console>:34
bookings = /media/sf_VirtualFolder/amadeus/bookings.csv.bz2 MapPartitionsRDD[3] at textFile at <console>:35
headerColumns_searches = List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Car...


List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Carrier, Seg6BookingCode, From, IsPublishedForNeg, IsFromInternet, IsFromVista, TerminalID, InternetOffice)

In [4]:
head_bookings

act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      

In [5]:
val searches_count = searches.count()

[Stage 4:=====================================================>   (14 + 1) / 15]

searches_count = 20390199


20390199

In [8]:
val bookings_count = bookings.count()

[Stage 6:=====================================================>   (16 + 1) / 17]

bookings_count = 10000011


10000011

## 2) Top 10 arrival airports in the world in 2013 (using the bookings file)
Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

In [8]:
/*Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineBookings(line:String) = {
    val fields   = line.split("\\^")
    //check if the line is complete..
    if (fields.length>34){
    val arr_port = fields(12).trim
    val paxstring = fields(34).trim
    if (arr_port=="" || paxstring=="") ("KKK","0") else
    (arr_port, paxstring)
    }
    else ("KKK","0")
}

parseLineBookings: (line: String)(String, String)


In [24]:
//val pair_rdd = bookings.mapPartitionsWithIndex{(idx, iter) => if (idx == 0) iter.drop(1) else iter}.map(parseLineBookings).cache()

Name: Syntax Error.
Message: 
StackTrace: 

In [9]:
val pair_rdd = bookings.map(parseLineBookings).persist(MEMORY_AND_DISK)

pair_rdd = MapPartitionsRDD[9] at map at <console>:37


MapPartitionsRDD[9] at map at <console>:37

In [10]:
pair_rdd.take(10)

[(arr_port,pax), (LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2)]

In [20]:
val pair_rdd_2 = pair_rdd.filter(_._1 != "arr_port" ).persist(MEMORY_AND_DISK)

pair_rdd_2 = MapPartitionsRDD[12] at filter at <console>:39


MapPartitionsRDD[12] at filter at <console>:39

In [16]:
pair_rdd_2.groupByKey

ShuffledRDD[11] at groupByKey at <console>:42

In [23]:
val kk = pair_rdd.filter(_._1 != "arr_port" ).persist(MEMORY_AND_DISK).filter(_._1 == "KKK").collect()

[Stage 8:=====================================================>   (16 + 1) / 17]

kk = Array((KKK,0))


[(KKK,0)]

In [24]:
val pair_rdd_good = pair_rdd.filter(_._1 != "arr_port" ).filter(_._1 != "KKK").persist(MEMORY_AND_DISK)

pair_rdd_good = MapPartitionsRDD[17] at filter at <console>:39


MapPartitionsRDD[17] at filter at <console>:39

In [25]:
pair_rdd_good.count()

[Stage 9:=====================================================>   (16 + 1) / 17]

10000009

In [26]:
def toInt(s: String): Option[Int] = {
  try {
    Some(s.toInt)
  } catch {
    case e: Exception => None
  }
}

toInt: (s: String)Option[Int]


In [27]:
//Make sure wee have integers
val pair_bookings = pair_rdd_good.map(x=> {
      val tst = toInt(x._2)
      tst  match {
      case None => (x._1, 0)
      case Some(y) => (x._1, y) }
}).persist(MEMORY_AND_DISK)

pair_bookings = MapPartitionsRDD[18] at map at <console>:43


MapPartitionsRDD[18] at map at <console>:43

In [28]:
pair_bookings.take(10)

[(LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2), (SIN,2)]

In [29]:
val temp = pair_bookings.reduceByKey(_+_)

temp = ShuffledRDD[19] at reduceByKey at <console>:45


ShuffledRDD[19] at reduceByKey at <console>:45

In [30]:
temp.take(10)

[Stage 11:====================================================>   (16 + 1) / 17]

[(IXS,390), (ELS,1370), (YQG,160), (SUV,220), (LFW,370), (DJE,1790), (PCL,210), (SBP,300), (DAL,1390), (TLC,430)]

In [33]:
val top10 = temp.map(x => (x._2,x._1)).sortByKey(false).take(10)

top10 = Array((88809,LHR), (70930,MCO), (70530,LAX), (69630,LAS), (66270,JFK), (64490,CDG), (59460,BKK), (58150,MIA), (58000,SFO), (55590,DXB))


[(88809,LHR), (70930,MCO), (70530,LAX), (69630,LAS), (66270,JFK), (64490,CDG), (59460,BKK), (58150,MIA), (58000,SFO), (55590,DXB)]

In [35]:
top10.foreach(e => println(e))

(88809,LHR)
(70930,MCO)
(70530,LAX)
(69630,LAS)
(66270,JFK)
(64490,CDG)
(59460,BKK)
(58150,MIA)
(58000,SFO)
(55590,DXB)


In [23]:
val arr_port_orderedByPax = mypair.sortByKey(false).take(10)

[Stage 9:=================>                                        (5 + 4) / 17]

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 9.0 failed 1 times, most recent failure: Lost task 8.0 in stage 9.0 (TID 63, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 34
	at $line47.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.parseLineBookings(<console>:36)
	at $line60.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:37)
	at $line60.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:37)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:216)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:957)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:948)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:888)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:948)
	at org

In [24]:
arr_port_orderedByPax

Name: Unknown Error
Message: lastException: Throwable = null
<console>:29: error: not found: value arr_port_orderedByPax
       arr_port_orderedByPax
       ^

StackTrace: 

In [5]:
rdd_bookings.count()

[Stage 2:=====================================================>   (16 + 1) / 17]

10000010

In [42]:
val kk = rdd_bookings.map(entry=>{
    val arr = entry(12);
    val pax = entry(34);
    (arr,pax)
}).persist(DISK_ONLY)
val kk2 = kk.filter(_._1 != " ").filter(_._2 != null) 
val kk3 = kk2.map{case (key,value) => (key.toString, value.toInt)}.persist(DISK_ONLY)

kk = MapPartitionsRDD[39] at map at <console>:39
kk2 = MapPartitionsRDD[41] at filter at <console>:44
kk3 = MapPartitionsRDD[42] at map at <console>:45


lastException: Throwable = null


MapPartitionsRDD[42] at map at <console>:45

In [43]:
val kk3 = kk.groupByKey().persist(DISK_ONLY)

kk3 = ShuffledRDD[43] at groupByKey at <console>:38


ShuffledRDD[43] at groupByKey at <console>:38

In [44]:
kk3.take(20)

[Stage 19:==========================>                              (8 + 2) / 17]

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 19.0 failed 1 times, most recent failure: Lost task 8.0 in stage 19.0 (TID 109, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 34
	at $line183.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:41)
	at $line183.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:39)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.serializer.SerializationStream.writeAll(Serializer.scala:141)
	at org.apache.spark.serializer.SerializerManager.dataSerializeStream(SerializerManager.scala:171)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1$$anonfun$apply$8.apply(BlockManager.scala:992)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1$$anonfun$apply$8.apply(BlockManager.scala:991)
	at org.apache.spark.storage.DiskStore.put(DiskStore.scala:57)
	at org.apache.spark.storage.BlockManager$$anonf

In [5]:
def ind_bookings(field: String): Int = headerColumns_bookings.indexOf(field) 
println(ind_bookings("arr_port"))
println(ind_bookings("pax"))

ind_bookings: (field: String)Int


12
34


In [6]:
val df_pair_ex1 = rdd_bookings.map(l => (l(12),l(34)))

df_pair_ex1 = MapPartitionsRDD[8] at map at <console>:39


MapPartitionsRDD[8] at map at <console>:39

In [7]:
val ex1 = df_pair_ex1.filter((_._2.toInt < 0) ).filter(_._2.toInt >= 0).groupByKey

ex1 = ShuffledRDD[11] at groupByKey at <console>:41


ShuffledRDD[11] at groupByKey at <console>:41

In [8]:
val ex11 = ex1.mapValues(v=>(v.map(x => x.toInt)).sum)

ex11 = MapPartitionsRDD[12] at mapValues at <console>:43


MapPartitionsRDD[12] at mapValues at <console>:43

In [9]:
val ex12 = ex11.take(10)

[Stage 4:===========================>                              (8 + 1) / 17]

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 4.0 failed 1 times, most recent failure: Lost task 8.0 in stage 4.0 (TID 42, localhost, executor driver): java.lang.IndexOutOfBoundsException: 34
	at scala.collection.LinearSeqOptimized$class.apply(LinearSeqOptimized.scala:65)
	at scala.collection.immutable.List.apply(List.scala:84)
	at $line44.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:39)
	at $line44.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:39)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$13.hasNext(Iterator.scala:462)
	at scala.collection.Iterator$$anon$13.hasNext(Iterator.scala:461)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:126)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.sca

## Check reading was correct

In [53]:
println(rdd_searches.first().length==head_searches.length)
println(head_searches)
println(rdd_searches.first())

true
List(date, time, txncode, officeid, country, origin, destination, roundtrip, nbsegments, seg1departure, seg1arrival, seg1date, seg1carrier, seg1bookingcode, seg2departure, seg2arrival, seg2date, seg2carrier, seg2bookingcode, seg3departure, seg3arrival, seg3date, seg3carrier, seg3bookingcode, seg4departure, seg4arrival, seg4date, seg4carrier, seg4bookingcode, seg5departure, seg5arrival, seg5date, seg5carrier, seg5bookingcode, seg6departure, seg6arrival, seg6date, seg6carrier, seg6bookingcode, from, ispublishedforneg, isfrominternet, isfromvista, terminalid, internetoffice)
List(2013-01-01, 20:25:57, MPT, 624d8c3ac0b3a7ca03e3c167e0f48327, DE, TXL, AUH, 1, 2, TXL, AUH, 2013-01-26, D2, , AUH, TXL, 2013-02-02, D2, , , , , , , , , , , , , , , , , , , , , , 1ASIWS, 0, 0, 0, d41d8cd98f00b204e9800998ecf8427e, FRA)


In [55]:
println(rdd_bookings.first().length==head_bookings.length)
println(head_bookings)
println(rdd_bookings.first())

true
List(act_date, source, pos_ctry, pos_iata, pos_oid, rloc, cre_date, duration, distance, dep_port, dep_city, dep_ctry, arr_port, arr_city, arr_ctry, lst_port, lst_city, lst_ctry, brd_port, brd_city, brd_ctry, off_port, off_city, off_ctry, mkt_port, mkt_city, mkt_ctry, intl, route, carrier, bkg_class, cab_class, brd_time, off_time, pax, year, month, oid)
List(2013-03-05 00:00:00, 1A, DE, a68dd7ae953c8acfb187a1af2dcbe123, 1a11ae49fcbf545fd2afc1a24d88d2b7, ea65900e72d71f4626378e2ebd298267, 2013-02-22 00:00:00, 1708, 0, ZRH, ZRH, CH, LHR, LON, GB, ZRH, ZRH, CH, LHR, LON, GB, ZRH, ZRH, CH, LHRZRH, LONZRH, CHGB, 1, LHRZRH, VI, T, Y, 2013-03-07 08:50:00, 2013-03-07 11:33:37, -1, 2013, 3, NULL)


## 1) Count number of lines on files

In [56]:
//Print number of lines
println(searches_count)
println(bookings_count)

20390199
10000011


This takes a long time to run (about 4 minutes each counting)

## I have some problem with the code above and It is a bit inefficient to work with such a big RDD, instead we would read directly the columns we need for each task.

## New approach

1) Read first rows

In [3]:
val first_searches = sc.textFile(searches_path).first//Take only first row and c
val first_bookings = sc.textFile(bookings_path).first//Take only first row and c

first_searches = Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice
first_bookings = "act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^o...


act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      

We can see that the separator is the character "\\^". So now we will create a list with the trimmed field names.

In [3]:
val lf_searches = first_searches.split("\\^").map(field=>field.trim).to[List]
val lf_bookings = first_bookings.split("\\^").map(field=>field.trim).to[List]

lf_searches = List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Carrier, Seg6BookingCode, From, IsPublishedForNeg, IsFromInternet, IsFromVista, TerminalID, InternetOffice)
lf_bookings = List(act_date, source, pos_ctry, pos_iata, pos_oid, rloc, cre_date, duration, distance, dep_port, dep_city, dep_ctry, arr_port, arr_city, arr_ctry, lst_port, lst_...


List(act_date, source, pos_ctry, pos_iata, pos_oid, rloc, cre_date, duration, distance, dep_port, dep_city, dep_ctry, arr_port, arr_city, arr_ctry, lst_port, lst_city, lst_ctry, brd_port, brd_city, brd_ctry, off_port, off_city, off_ctry, mkt_port, mkt_city, mkt_ctry, intl, route, carrier, bkg_class, cab_class, brd_time, off_time, pax, year, month, oid)

Now I will create a dictionary to be able to refer to the column just by using the field name.

In [4]:
val dsearch: Map[String, Int] = Map(lf_searches map {s => (s, lf_searches.indexOf(s))} : _*);//searchs dictionary
val dbook: Map[String, Int] = Map(lf_bookings map {s => (s, lf_bookings.indexOf(s))} : _*);//bookings dictionary

dsearch = Map(TxnCode -> 2, Seg6Departure -> 34, TerminalID -> 43, Country -> 4, Seg1Carrier -> 12, Seg3Departure -> 19, Seg5Departure -> 29, Origin -> 5, RoundTrip -> 7, OfficeID -> 3, Seg2Arrival -> 15, InternetOffice -> 44, Seg5Carrier -> 32, Seg3Date -> 21, Seg5BookingCode -> 33, Seg2Departure -> 14, Seg1BookingCode -> 13, Seg4Departure -> 24, Seg6Carrier -> 37, Seg6Arrival -> 35, IsFromVista -> 42, Seg4Date -> 26, IsPublishedForNeg -> 40, Seg4Arrival -> 25, IsFromInternet -> 41, Seg1Arrival -> 10, Date -> 0, Seg2Carrier -> 17, Seg4Carrier -> 27, Seg6BookingCode -> 38, Seg3Arrival -> 20, NbSegments -> 8, Destination -> 6, Seg1Departure -> 9, Seg1Date -> 11, Seg3BookingCode -> 23, Seg3Carrier -> 22, Seg5Date -> 31, Seg2BookingCode -> 18, Seg5Arrival -> 30, Seg6Date -...


Map(TxnCode -> 2, Seg6Departure -> 34, TerminalID -> 43, Country -> 4, Seg1Carrier -> 12, Seg3Departure -> 19, Seg5Departure -> 29, Origin -> 5, RoundTrip -> 7, OfficeID -> 3, Seg2Arrival -> 15, InternetOffice -> 44, Seg5Carrier -> 32, Seg3Date -> 21, Seg5BookingCode -> 33, Seg2Departure -> 14, Seg1BookingCode -> 13, Seg4Departure -> 24, Seg6Carrier -> 37, Seg6Arrival -> 35, IsFromVista -> 42, Seg4Date -> 26, IsPublishedForNeg -> 40, Seg4Arrival -> 25, IsFromInternet -> 41, Seg1Arrival -> 10, Date -> 0, Seg2Carrier -> 17, Seg4Carrier -> 27, Seg6BookingCode -> 38, Seg3Arrival -> 20, NbSegments -> 8, Destination -> 6, Seg1Departure -> 9, Seg1Date -> 11, Seg3BookingCode -> 23, Seg3Carrier -> 22, Seg5Date -> 31, Seg2BookingCode -> 18, Seg5Arrival -> 30, Seg6Date -> 36, From -> 39, Time -> 1, Seg4BookingCode -> 28, Seg2Date -> 16)

In [5]:
dbook("pax")

34

In [6]:
dbook("arr_port")

12

Now we can read bookings file selecting for instance the columns of arr_port and pax.

In [4]:
import org.apache.spark.{RangePartitioner,HashPartitioner, SparkContext}

Nice! now we can use the transformations and methods for Pair RDDs!!

In [7]:
val root_path: String = "/media/sf_VirtualFolder/amadeus/";//where to find files
val searches_path: String = root_path + "searches.csv.bz2";
val bookings_path: String = root_path + "bookings.csv.bz2";
//Now drop first line, split lines with ^ separator and trim words
val head1 = sc.textFile(bookings_path).first
val rdd1 = sc.textFile(bookings_path).filter(_!=head1).map(line => (line.split("\\^")(12).trim, line.split("\\^")(34).toInt)).persist()
val rdd2 = rdd1.filter(_._2!="").filter(!_._1.isEmpty).filter(_._2!=null).filter(_._1!=null)
//rdd1.count()
rdd2.count()

head1 = "act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      "
rdd1 = MapPartitionsRDD[15] at map at <console>:40


MapPartitionsRDD[15] at map at <console>:40

In [8]:
val rdd2 = rdd1.filter(_._2!="").filter(!_._1.isEmpty).filter(_._2!=null).filter(_._1!=null)

rdd2 = MapPartitionsRDD[19] at filter at <console>:39


MapPartitionsRDD[19] at filter at <console>:39

In [9]:
rdd2.take(10)

[(LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2), (SIN,2)]

In [11]:
rdd1.count()

[Stage 6:=================>                                        (5 + 4) / 17]

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 6.0 failed 1 times, most recent failure: Lost task 8.0 in stage 6.0 (TID 35, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 34
	at $line46.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$2.apply(<console>:40)
	at $line46.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$2.apply(<console>:40)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:216)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:957)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:948)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:888)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:948)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:694)
	at org

In [30]:
//val rdd3 = rdd1.filter(_._1!=null).filter(_._2!= null).map(x=>(x._1.trim().toUpperCase(), x._2.toInt)).reduceByKey(_+_)
val rdd3 = rdd1.filter(_._1.length>0).filter(_._2!=null)
//rdd1.filter(_._1 !=null).foreach(println)
//val kk = rdd3.filter(_._2>10).take(10)

//val rdd4 = rdd2.reduceByKey((x,y) => x + y )
//.mapByValue(_+_)
//.groupByKey
//
//rdd4.take(10)
//val top10 = rdd3.sortByKey(false).take(10)

[Stage 22:======>                                                  (2 + 8) / 17]

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 22.0 failed 1 times, most recent failure: Lost task 8.0 in stage 22.0 (TID 82, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
StackTrace:   at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
  at scala.Option.foreach(Option.scala:257)
  at org.apache.spark.scheduler.D

In [ ]:
top10.forEach(println(_))

In [26]:
rdd3.max

[Stage 17:===>                                                     (1 + 8) / 17]

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 8 in stage 17.0 failed 1 times, most recent failure: Lost task 8.0 in stage 17.0 (TID 57, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
StackTrace: Driver stacktrace:
  at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
  at org.apache.spark.sch

## 2) Top 10 arrival airports in the world in 2013 (using the bookings file)
Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)